# Regenerate GVCF Files
```
pi:ababaian
files: ~/Crown/.. (var0
start: 2019 06 19
complete : YYYY MM DD
```
## Introduction

I have been using the `ADcalc*.sh` script to create a common set of GVCF files for downstream analysis. The TCGA and CCLE data was proccessed uniformly.

I will re-process the CRC and hCAGE data for cell lines and primary cells to make GVCF files for analysis. This will make cross cohort analysis simplified.



In [1]:
cd ~/Crown/data2/hCAGE/
ls

2011_hCAGE_protocol.pdf       ADcalc_cage.sh  hCAGE_gvcf.log  screenlog.0
2014_MORAI_workflow_CAGE.pdf  cell_lines      primary         TssHmm_Info.pdf


## Objective

1. Generate `GVCF` files for hCAGE, CRC1 and CRC4 data-sets



## Materials and Methods -- hCAGE

Launch an AMI instance and run ADcalc script on data.


### Initialization

Initialize data on AMI instance, run ADcalc and output data to respective folder

In [ ]:
# DNS:ec2-34-221-202-76.us-west-2.compute.amazonaws.com
# AMI: ami-0b375c9c58cb4a7a2 (TCGA aligner)
# Instance: m4.4xlarge
# Storage: 400 Gb

# ON REMOTE:

## Copy hCAGE files into it's dir
#mkdir -p ~/hCAGE; cd hCAGE;
#aws s3 cp --recursive s3://crownproject/hCAGE/ ./

## PATHS:
## ~/hCAGE/cell_lines/BAM/*
## ~/hCAGE/primary/BAM/*

# cd ~/hCAGE/cell_lines/BAM/

## Run ADcalc script.sh
# screen -L
# bash ADcalc_tcga.sh

# aws s3 cp screenlog.0 s3://crownproject/ccle/logs/ccle.gvcf.log

aws s3 cp s3://crownproject/ccle/logs/ccle.gvcf.log ./
cat ccle.gvcf.log

## DONE

### Scripts
`ADcalc_hCAGE.sh` @ s3://crownproject/hCAGE/ADcalc_hCAGE.sh


In [1]:
#!/bin/bash
# ADcalc_hCAGE.sh
# Allelic Depth Calculator
# for a position
#
# s3://crownproject/hCAGE/ADcalc_hcage.sh

# Controls -----------------
DEPTH='100000' #Max per file DP

# Regions in hgr1.fa reference genome
REGIONS=('chr13:1003660-1005529' 'chr13:1005529-1005629' \
        'chr13:10219-10340' 'chr13:1006622-1006779' 'chr13:1007948-1013018')

# Corresponding region/gene names
GENES=('18S' '18SE' '5S' '5.8S' '28S')

# 18S  1870
# 18SE 101
# 5S   122
# 5.8S 158
# 28S  5071

# Terminate instances upon completion (for debugging)
TERMINATE='FALSE'

# S3 Output directory
S3DIR='s3://crownproject/hCAGE/gvcf/'
BAMLIST='bam.list.tmp'

# Script ------------------ ------------------------------
cd ~/hCAGE/cell_lines/
mkdir -p GVCF #Output Folder
TYPE='hCAGE_cell_lines' # hardcode single ccle run
cd BAM

#for TYPE in $(echo "hgr1")
#do
    echo Analyzing $TYPE...
    #cd $TYPE

    ls *.bam > bam.list.tmp
    ls *.bam > ../GVCF/$TYPE.bamlist
          
    for index in ${!GENES[*]}
    do
      printf "Started processing %s\n" ${GENES[$index]}
      OUTPUT="../GVCF/$TYPE.${GENES[$index]}.gvcf"

      # Iterate through every bam file in directory
      # look-up position and return VCF
      bcftools mpileup -f ~/resources/hgr1/hgr1.fa \
        --max-depth $DEPTH -A --min-BQ 30 \
        -a FORMAT/DP,AD \
        -r ${REGIONS[$index]} \
        --ignore-RG \
        -b $BAMLIST | \
        bcftools annotate -x INFO,FORMAT/PL - | \
        bcftools view -O v - \
        >> $OUTPUT

      RESULTS+=("$OUTPUT")
      printf "Done with %s \n" ${GENES[$index]}
      printf "%s\n" ${REGIONS[$index]}

    done

    rm bam.list.tmp

#    cd .. # move to tcga folder to reset
#done

# Copy GVCF output to AWS S3
cd ../GVCF
aws s3 cp --recursive ./ $S3DIR

# -------------------------------------------
cd ~/hCAGE/primary/
mkdir -p GVCF #Output Folder
TYPE='hCAGE_cell_lines' # hardcode single ccle run
cd BAM

#for TYPE in $(echo "hgr1")
#do
    echo Analyzing $TYPE...
    #cd $TYPE

    ls *.bam > bam.list.tmp
    ls *.bam > ../GVCF/$TYPE.bamlist
          
    for index in ${!GENES[*]}
    do
      printf "Started processing %s\n" ${GENES[$index]}
      OUTPUT="../GVCF/$TYPE.${GENES[$index]}.gvcf"

      # Iterate through every bam file in directory
      # look-up position and return VCF
      bcftools mpileup -f ~/resources/hgr1/hgr1.fa \
        --max-depth $DEPTH -A --min-BQ 30 \
        -a FORMAT/DP,AD \
        -r ${REGIONS[$index]} \
        --ignore-RG \
        -b $BAMLIST | \
        bcftools annotate -x INFO,FORMAT/PL - | \
        bcftools view -O v - \
        >> $OUTPUT

      RESULTS+=("$OUTPUT")
      printf "Done with %s \n" ${GENES[$index]}
      printf "%s\n" ${REGIONS[$index]}

    done

    rm bam.list.tmp

#    cd .. # move to tcga folder to reset
#done

# Copy GVCF output to AWS S3
cd ../GVCF
aws s3 cp --recursive ./ $S3DIR


# Shutdown and Terminate instance
EC2ID=$(ec2metadata --instance-id)
sleep 20s # to catch errors

if [ "$TERMINATE" = TRUE ]
then
  echo "Run Complete -- Shutting down instance."
  aws ec2 terminate-instances --instance-ids $EC2ID
else
  echo "Run Complete -- Instance is online."
fi


upload: ./00_TCGA_Run_template.ipynb to s3://crownproject/hCAGE/gvcf/00_TCGA_Run_template.ipynb
upload: .ipynb_checkpoints/00_TCGA_Run_template-checkpoint.ipynb to s3://crownproject/hCAGE/gvcf/.ipynb_checkpoints/00_TCGA_Run_template-checkpoint.ipynb
upload: .ipynb_checkpoints/20190618_RE_GVCF-checkpoint.ipynb to s3://crownproject/hCAGE/gvcf/.ipynb_checkpoints/20190618_RE_GVCF-checkpoint.ipynb
upload: .ipynb_checkpoints/20190506_TCGA_Run2_Pilot-checkpoint.ipynb to s3://crownproject/hCAGE/gvcf/.ipynb_checkpoints/20190506_TCGA_Run2_Pilot-checkpoint.ipynb
upload: .ipynb_checkpoints/20190507_Crown_EC2_Update-checkpoint.ipynb to s3://crownproject/hCAGE/gvcf/.ipynb_checkpoints/20190507_Crown_EC2_Update-checkpoint.ipynb
upload: ./00_template.ipynb to s3://crownproject/hCAGE/gvcf/00_template.ipynb
upload: .ipynb_checkpoints/Untitled-checkpoint.ipynb to s3://crownproject/hCAGE/gvcf/.ipynb_checkpoints/Untitled-checkpoint.ipynb
upload: .ipynb_checkpoints/20190509_TCGA_gvcf-checkpoint.ipynb to s3:/

In [3]:
# aws s3 cp screenlog.0 s3://crownproject/hCAGE/hCAGE_gvcf.log

aws s3 cp s3://crownproject/hCAGE/hCAGE_gvcf.log ./

cat hCAGE_gvcf.log

## shit, the primary cell data was misnamed (line 84) as cell line and overwrote
## the original run. Will have to re-run for cell lines and rename primary hCAGE data
# on AWS now

download: s3://crownproject/hCAGE/hCAGE_gvcf.log to ./hCAGE_gvcf.log
































































































































































































In [4]:
mkdir -p gvcf
aws s3 cp --recursive s3://crownproject/hCAGE/gvcf/ ./gvcf/

download: s3://crownproject/hCAGE/gvcf/hCAGE_cell_lines.5S.gvcf to gvcf/hCAGE_cell_lines.5S.gvcf
download: s3://crownproject/hCAGE/gvcf/hCAGE_cell_lines.bamlist to gvcf/hCAGE_cell_lines.bamlist
download: s3://crownproject/hCAGE/gvcf/hCAGE_cell_lines.18SE.gvcf to gvcf/hCAGE_cell_lines.18SE.gvcf
download: s3://crownproject/hCAGE/gvcf/hCAGE_primary.5.8S.gvcf to gvcf/hCAGE_primary.5.8S.gvcf
download: s3://crownproject/hCAGE/gvcf/hCAGE_primary.18SE.gvcf to gvcf/hCAGE_primary.18SE.gvcf
download: s3://crownproject/hCAGE/gvcf/hCAGE_cell_lines.5.8S.gvcf to gvcf/hCAGE_cell_lines.5.8S.gvcf
download: s3://crownproject/hCAGE/gvcf/hCAGE_primary.5S.gvcf to gvcf/hCAGE_primary.5S.gvcf
download: s3://crownproject/hCAGE/gvcf/hCAGE_primary.bamlist to gvcf/hCAGE_primary.bamlist
download: s3://crownproject/hCAGE/gvcf/hCAGE_cell_lines.18S.gvcf to gvcf/hCAGE_cell_lines.18S.gvcf
download: s3://crownproject/hCAGE/gvcf/hCAGE_primary.18S.gvcf to gvcf/hCAGE_primary.18S.gvcf
download: s3://crownproject/hCAGE/gvcf/h

## Materials and Methods -- CRC

Launch an AMI instance and run ADcalc script on data.

In [ ]:
# DNS:XXX
# AMI: ami-0b375c9c58cb4a7a2 (TCGA aligner)
# Instance: m4.4xlarge
# Storage: 400 Gb

# ON REMOTE:

## Copy CRC files into it's dir
#mkdir -p ~/crc; cd crc;
#aws s3 cp --recursive s3://crownproject/crc/ ./

## PATHS:
## ~/hCAGE/cell_lines/BAM/*
## ~/hCAGE/primary/BAM/*

# cd ~/hCAGE/cell_lines/BAM/

## Run ADcalc script.sh
# screen -L
# bash ADcalc_tcga.sh

# aws s3 cp screenlog.0 s3://crownproject/ccle/logs/ccle.gvcf.log

aws s3 cp s3://crownproject/ccle/logs/ccle.gvcf.log ./
cat ccle.gvcf.log

## DONE

In [ ]:
#!/bin/bash
# ADcalc_crc.sh
# Allelic Depth Calculator
# for a position
#
# s3://crownproject/hCAGE/ADcalc_crc.sh

# Controls -----------------
DEPTH='100000' #Max per file DP

# Regions in hgr1.fa reference genome
REGIONS=('chr13:1003660-1005529' 'chr13:1005529-1005629' \
        'chr13:10219-10340' 'chr13:1006622-1006779' 'chr13:1007948-1013018')

# Corresponding region/gene names
GENES=('18S' '18SE' '5S' '5.8S' '28S')

# 18S  1870
# 18SE 101
# 5S   122
# 5.8S 158
# 28S  5071

# Terminate instances upon completion (for debugging)
TERMINATE='FALSE'

# S3 Output directory
S3DIR='s3://crownproject/crc/gvcf/'
BAMLIST='bam.list.tmp'

# Script ------------------ ------------------------------
cd ~/crc/
mkdir -p GVCF #Output Folder
TYPE='crc1' # hardcode single ccle run
cd crc1/hgr1/

#for TYPE in $(echo "hgr1")
#do
    echo Analyzing $TYPE...
    #cd $TYPE

    ls *.bam > bam.list.tmp
    ls *.bam > ../../GVCF/$TYPE.bamlist
          
    for index in ${!GENES[*]}
    do
      printf "Started processing %s\n" ${GENES[$index]}
      OUTPUT="../../GVCF/$TYPE.${GENES[$index]}.gvcf"

      # Iterate through every bam file in directory
      # look-up position and return VCF
      bcftools mpileup -f ~/resources/hgr1/hgr1.fa \
        --max-depth $DEPTH -A --min-BQ 30 \
        -a FORMAT/DP,AD \
        -r ${REGIONS[$index]} \
        --ignore-RG \
        -b $BAMLIST | \
        bcftools annotate -x INFO,FORMAT/PL - | \
        bcftools view -O v - \
        >> $OUTPUT

      RESULTS+=("$OUTPUT")
      printf "Done with %s \n" ${GENES[$index]}
      printf "%s\n" ${REGIONS[$index]}

    done

    rm bam.list.tmp

#    cd .. # move to tcga folder to reset
#done

# Copy GVCF output to AWS S3
cd ../../GVCF
aws s3 cp --recursive ./ $S3DIR

# -------------------------------------------
cd ~/crc/
mkdir -p GVCF #Output Folder
TYPE='crc4' # hardcode single ccle run
cd crc4/hgr1/

#for TYPE in $(echo "hgr1")
#do
    echo Analyzing $TYPE...
    #cd $TYPE

    ls *.bam > bam.list.tmp
    ls *.bam > ../../GVCF/$TYPE.bamlist
          
    for index in ${!GENES[*]}
    do
      printf "Started processing %s\n" ${GENES[$index]}
      OUTPUT="../../GVCF/$TYPE.${GENES[$index]}.gvcf"

      # Iterate through every bam file in directory
      # look-up position and return VCF
      bcftools mpileup -f ~/resources/hgr1/hgr1.fa \
        --max-depth $DEPTH -A --min-BQ 30 \
        -a FORMAT/DP,AD \
        -r ${REGIONS[$index]} \
        --ignore-RG \
        -b $BAMLIST | \
        bcftools annotate -x INFO,FORMAT/PL - | \
        bcftools view -O v - \
        >> $OUTPUT

      RESULTS+=("$OUTPUT")
      printf "Done with %s \n" ${GENES[$index]}
      printf "%s\n" ${REGIONS[$index]}

    done

    rm bam.list.tmp

#    cd .. # move to tcga folder to reset
#done

# Copy GVCF output to AWS S3
cd ../../GVCF
aws s3 cp --recursive ./ $S3DIR


# Shutdown and Terminate instance
EC2ID=$(ec2metadata --instance-id)
sleep 20s # to catch errors

if [ "$TERMINATE" = TRUE ]
then
  echo "Run Complete -- Shutting down instance."
  aws ec2 terminate-instances --instance-ids $EC2ID
else
  echo "Run Complete -- Instance is online."
fi


## Discussion
